In [3]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [4]:
!pip install keras

^C


In [4]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


Using legacy 'setup.py install' for termcolor, since package 'wheel' is not installed.
Using legacy 'setup.py install' for wrapt, since package 'wheel' is not installed.
  Attempting uninstall: h5py
    Found existing installation: h5py 3.3.0
    Uninstalling h5py-3.3.0:
      Successfully uninstalled h5py-3.3.0
    Running setup.py install for termcolor: started
    Running setup.py install for termcolor: finished with status 'done'
    Running setup.py install for wrapt: started
    Running setup.py install for wrapt: finished with status 'done'



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [5]:
words_set = set()
word_to_vec = {}
word_to_idx = {}

In [6]:
with open('glove.6B.50d.txt', 'r', encoding='utf8') as fr:
    for line in fr:
        line = line.strip().split()
        curr_w = line[0]
        word_to_vec[curr_w] = np.array(line[1:], dtype=np.float64)
        words_set.add(curr_w)
    i = 1
    for word in sorted(words_set):
        word_to_idx[word] = i
        i += 1

In [7]:
word_to_idx["hi"]

178126

In [8]:
word_to_vec['hi']

array([-0.54313  ,  0.34427  ,  0.27125  ,  1.0487   , -1.1642   ,
       -1.2722   ,  0.35781  , -0.56527  , -0.29879  ,  0.85179  ,
        0.52222  , -0.0019718, -0.46435  ,  0.033631 ,  0.048367 ,
        0.78762  ,  0.075995 ,  0.51577  ,  0.34778  ,  0.53802  ,
        0.28299  , -0.1313   , -0.073753 ,  0.42614  ,  0.030954 ,
       -0.55033  , -0.99789  , -0.28947  ,  0.30517  , -1.1194   ,
        1.2957   ,  0.91165  ,  0.32222  ,  0.93405  , -0.34152  ,
       -0.62713  , -0.092165 ,  0.50901  ,  0.29204  , -0.20122  ,
        0.19614  , -0.45882  ,  1.1099   , -0.68737  ,  1.5724   ,
       -0.10446  ,  0.23594  , -0.56594  ,  0.43676  ,  0.98093  ])

In [62]:
emoji_mapping = {"0": ":heart:", "1": ":ball:", "2": ":smile:", "3": ":disappointed:", "4":":fork_and_knife:"}

In [9]:
def process_sentence(X, word_to_idx, max_len):
    num_training_samples = X.shape[0]
    #pad each sentence with zeros to make each sequence of equal length
    X_processed = np.zeros((num_training_samples, max_len))
    
    for i in range(num_training_samples):
        words = X[i].lower().split()
        j = 0
        for word in words:
            #replace word by their index in our vocabulary, embedding of which is already known
            X_processed[i,j] = word_to_idx[word]
            j += 1
    
    return X_processed

In [10]:
#now the embedding layer(pre-trained)
def embed_layer(word_to_vec, word_to_idx):
    vocab_size = len(word_to_idx)+1
    embVec_dim = word_to_vec['sample'].shape[0]
    embed_matrix = np.zeros((vocab_size, embVec_dim))
    for k,v in word_to_idx.items():
        embed_matrix[v,] = word_to_vec[k]
        
    embedding_layer = Embedding(vocab_size, embVec_dim, trainable=False)#static embedding vectors
    embedding_layer.build((None,))
    embedding_layer.set_weights([embed_matrix])
    
    return embedding_layer

In [11]:
from keras.layers import Bidirectional

In [37]:
def lstm_model(input_shape, word_to_vec, word_to_idx):
    input_sentences = Input(shape=input_shape, dtype='int32')
    embedding_layer = embed_layer(word_to_vec, word_to_idx)
    embeddings = embedding_layer(input_sentences)
    
    X = Bidirectional(LSTM(128, return_sequences = True))(embeddings)
    X = Dropout(0.5)(X)
    X = Bidirectional(LSTM(128, return_sequences = False))(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation("softmax")(X)
    
    model = Model(inputs = input_sentences, outputs=X)
    
    return model

In [13]:
import csv

In [78]:
X_train = []
y_train = []
with open('train_emoji.csv') as data:
    csv_reader = csv.reader(data)
    for row in csv_reader:
        X_train.append(row[0])
        y_train.append(row[1])
X_train = np.asarray(X_train)
y_train = np.asarray(y_train, dtype=int)

X_test = []
y_test = []
with open('test_emoji.csv') as data:
    csv_reader = csv.reader(data)
    for row in csv_reader:
        X_test.append(row[0])
        y_test.append(row[1])
X_test = np.asarray(X_test)
y_test = np.asarray(y_test, dtype=int)

In [79]:
X_train[0]

'never talk to me again'

In [80]:
max_len_of_sentence = 0
for i in range(len(X_train)):
    max_len_of_sentence = max(max_len_of_sentence, len(X_train[i].split()))

In [81]:
print(max_len_of_sentence)

10


In [82]:
model = lstm_model((max_len_of_sentence,), word_to_vec, word_to_idx)

In [83]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [84]:
X_train_processed = process_sentence(X_train, word_to_idx, max_len_of_sentence)
y_train_oh = np.eye(5)[y_train.reshape(-1)]

In [85]:
X_val_processed = X_train_processed[-20:]
y_val_oh = y_train_oh[-20:]
X_train_processed = X_train_processed[:-20]
y_train_oh = y_train_oh[:-20]

In [33]:
import pandas as pd

In [86]:
model.fit(X_train_processed, y_train_oh, epochs=50, validation_data=(X_val_processed, y_val_oh), batch_size=32, shuffle=True)

Epoch 1/50
10/10 [==============================] - 7s 193ms/step - loss: 1.5859 - accuracy: 0.2126 - val_loss: 1.4533 - val_accuracy: 0.2000
Epoch 2/50
10/10 [==============================] - 0s 28ms/step - loss: 1.3892 - accuracy: 0.4275 - val_loss: 1.2282 - val_accuracy: 0.6500
Epoch 3/50
10/10 [==============================] - 0s 28ms/step - loss: 1.0931 - accuracy: 0.6607 - val_loss: 0.9946 - val_accuracy: 0.5500
Epoch 4/50
10/10 [==============================] - 0s 28ms/step - loss: 0.7852 - accuracy: 0.7255 - val_loss: 1.0786 - val_accuracy: 0.5500
Epoch 5/50
10/10 [==============================] - 0s 27ms/step - loss: 0.6613 - accuracy: 0.7524 - val_loss: 0.7091 - val_accuracy: 0.6500
Epoch 6/50
10/10 [==============================] - 0s 32ms/step - loss: 0.4476 - accuracy: 0.8463 - val_loss: 0.5854 - val_accuracy: 0.7000
Epoch 7/50
10/10 [==============================] - 0s 33ms/step - loss: 0.3641 - accuracy: 0.8931 - val_loss: 0.7612 - val_accuracy: 0.7000
Epoch 8/50
1

In [87]:
X_test_processed = process_sentence(X_test, word_to_idx, max_len_of_sentence)
y_test_oh = np.eye(5)[y_test.reshape(-1)]
loss, acc = model.evaluate(X_test_processed, y_test_oh)
print("Loss: ", loss, " Accuracy: ", acc)

2/2 [==============================] - 0s 7ms/step - loss: 0.9755 - accuracy: 0.8036
Loss:  0.9754543304443359  Accuracy:  0.8035714030265808


In [54]:
!pip install emoji

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for emoji: filename=emoji-1.4.1-py3-none-any.whl size=186373 sha256=0fe557eae3b83601d423956d2274d2d5e473f46d716e2c1c59b01104e43b4dc1
  Stored in directory: c:\users\ashutosh sharma\appdata\local\pip\cache\wheels\9e\a5\d8\dea02b6be962ee32404006efa68513e8701aabc12fd80fd3f6
Successfully built emoji


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [55]:
import emoji

In [64]:
def test_model(input_string):
    x_test = np.array([input_string])
    x_test_processed = process_sentence(x_test, word_to_idx, max_len_of_sentence)
    result = str(np.argmax(model.predict(x_test_processed)))
    print(x_test[0] + ' ' + emoji.emojize(emoji_mapping[result], use_aliases=True))

In [88]:
test_model('This is not great')

This is not great 😞


In [96]:
test_model('This is great')

This is great 😄


In [104]:
test_model('The dinner is delicious')

The dinner is delicious 🍴
